In [1]:
# 📦 1. Imports
import kagglehub
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization # Added BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay # Added
from tensorflow.keras.regularizers import l2 # Added
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score # Added accuracy_score & confusion_matrix
import numpy as np
import math # Added

# --- Configuration (Moved here for clarity) ---
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
N_EPOCHS = 25 # Increased epochs
VALIDATION_SPLIT = 0.2 # Using 20% of train data for validation
L2_REG = 1e-4 # Regularization factor
INITIAL_LR = 1e-4
LR_DECAY_STEPS_FACTOR = 5
LR_DECAY_RATE = 0.9
EARLY_STOPPING_PATIENCE = 15
REDUCE_LR_PATIENCE = 5

# 📂 2. Download Dataset
path = kagglehub.dataset_download("prashant268/chest-xray-covid19-pneumonia")
dataset = Path(path)
train_dir = dataset / "Data/train"
test_dir  = dataset / "Data/test"

# 📸 3. Preprocessing (Subtle Augmentation & Validation Split)
from tensorflow.keras.applications.vgg16 import preprocess_input

# Train data generator with subtle augmentation and validation split
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,         # Subtle augmentation
    brightness_range=[0.95, 1.05], # Subtle augmentation
    validation_split=VALIDATION_SPLIT
)

# Test/Validation data generator with only preprocessing
test_val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Training batches
train_batches = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    subset='training' # Specify training subset
)

# Validation batches
validation_batches = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False, # No shuffle for validation
    class_mode='categorical',
    subset='validation' # Specify validation subset
)

# Test batches
test_batches = test_val_datagen.flow_from_directory(
    directory=test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical'
)

# Calculate steps
steps_per_epoch_train = math.ceil(train_batches.samples / BATCH_SIZE)
steps_per_epoch_val = math.ceil(validation_batches.samples / BATCH_SIZE)
steps_per_epoch_test = math.ceil(test_batches.samples / BATCH_SIZE)
num_classes = train_batches.num_classes # Get number of classes

# 🧠 4. Model Definition (Added BN, Depth, Regularization)
model = Sequential([
    Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(L2_REG), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(L2_REG)),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(L2_REG)),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Added 4th Conv Block
    Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(L2_REG)),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, kernel_regularizer=l2(L2_REG)),
    BatchNormalization(), # Added BN
    tf.keras.layers.Activation('relu'),
    Dropout(0.5),

    Dense(128, kernel_regularizer=l2(L2_REG)), # Added regularization
    tf.keras.layers.Activation('relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
])
model.summary()

# 🧪 5. Compile Model (Added LR Schedule)
# Learning Rate Schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=INITIAL_LR,
    decay_steps=steps_per_epoch_train * LR_DECAY_STEPS_FACTOR,
    decay_rate=LR_DECAY_RATE,
    staircase=True
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), # Use schedule
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🛠 6. Callbacks (Monitoring val_loss)
callbacks = [
    ModelCheckpoint(
        'best_model_val_loss.keras', # Updated filename
        save_best_only=True,
        monitor='val_loss', # Monitor validation loss
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss', # Monitor validation loss
        patience=EARLY_STOPPING_PATIENCE, # Updated patience
        restore_best_weights=True,
        verbose=1
    )
]


# 🚀 7. Train Model (Added Validation Data)
history = model.fit(
    train_batches,
    epochs=N_EPOCHS, # Increased epochs
    steps_per_epoch=steps_per_epoch_train,
    validation_data=validation_batches, # Added validation data
    validation_steps=steps_per_epoch_val, # Added validation steps
    callbacks=callbacks
)

# The best model based on val_loss is automatically restored by EarlyStopping

# ✅ 8. Evaluate Model (On Test Set)
loss, acc = model.evaluate(test_batches, steps=steps_per_epoch_test) # Use test steps
print(f"\n✅ Test Loss: {loss:.4f}")
print(f"✅ Test Accuracy: {acc:.4f}")

# 📊 9. Classification Report & Confusion Matrix (On Test Set)
predictions = model.predict(test_batches, steps=steps_per_epoch_test) # Use test steps
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_batches.classes
labels = list(test_batches.class_indices.keys())

# Ensure correct number of samples are used for metrics
num_test_samples = test_batches.samples
true_classes = true_classes[:num_test_samples]
predicted_classes = predicted_classes[:num_test_samples]

# Calculate & print accuracy (using sklearn)
tta_accuracy = accuracy_score(true_classes, predicted_classes)
print(f"\n✅ Final Test Accuracy: {tta_accuracy:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\n🔍 Confusion Matrix:")
print(conf_matrix)

# Classification Report
print("\n📄 Classification Report:\n")
print(classification_report(true_classes, predicted_classes, target_names=labels))

# Print class-wise stats cleanly (optional, kept from your original additions)
report = classification_report(true_classes, predicted_classes, target_names=labels, output_dict=True)
print("\n📊 Class-wise Performance:")
for label in labels:
    if label in report:
        cls = report[label]
        print(f"   {label.upper()} — Precision: {cls['precision']:.4f}, Recall: {cls['recall']:.4f}, F1: {cls['f1-score']:.4f}")

Found 4116 images belonging to 3 classes.
Found 1028 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    12,845,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 13,269,955 (50.62 MB)

 Trainable params: 13,268,483 (50.62 MB)

 Non-trainable params: 1,472 (5.75 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 732ms/step - accuracy: 0.8157 - loss: 0.5756
Epoch 1: val_loss improved from inf to 0.77798, saving model to best_model_val_loss.keras
129/129 ━━━━━━━━━━━━━━━━━━━━ 133s 924ms/step - accuracy: 0.8162 - loss: 0.5746 - val_accuracy: 0.7374 - val_loss: 0.7780
Epoch 2/25
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - accuracy: 0.9300 - loss: 0.3101
Epoch 2: val_loss improved from 0.77798 to 0.33899, saving model to best_model_val_loss.keras
129/129 ━━━━━━━━━━━━━━━━━━━━ 92s 608ms/step - accuracy: 0.9300 - loss: 0.3100 - val_accuracy: 0.9183 - val_loss: 0.3390
Epoch 3/25
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step - accuracy: 0.9416 - loss: 0.2673
Epoch 3: val_loss improved from 0.33899 to 0.21422, saving model to best_model_val_loss.keras
129/129 ━━━━━━━━━━━━━━━━━━━━ 71s 548ms/step - accuracy: 0.9416 - loss: 0.2673 - val_accuracy: 0.9650 - val_loss: 0.2142
Epoch 4/25
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - accuracy: 0.9397 - loss: 0.2532
Epoch 4: 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os

folder_path = '/content/drive/MyDrive/My_Models'
os.makedirs(folder_path, exist_ok=True)

# Create path to your drive
model_save_path = '/content/drive/MyDrive/My_Models/chest_final_model.keras'

# Save the model
model.save(model_save_path)


Mounted at /content/drive
